Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [14]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


**Reshape THE IMAGE TO 1-D vector.**

In [15]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


Build the computation graph.
    - We will build neural network with one hidden layer and 1024 neurons in that hidden layer.

In [16]:
batch_size = 128
relu_counts = 1024
graph = tf.Graph()
with graph.as_default():
#     Prepare the data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
        tf.truncated_normal([image_size*image_size, relu_counts]))
    biases = tf.Variable(tf.zeros([relu_counts]))
    
    weights_two = tf.Variable(
        tf.truncated_normal([relu_counts, num_labels]))
    biases_two = tf.Variable(tf.zeros([num_labels]))
    
#     Construct relu neural network
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    tf_layer_two_input = tf.nn.relu(logits)
    
    logits_layer_two = tf.matmul(tf_layer_two_input, weights_two) + biases_two
    
#     Define Loss Function
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits_layer_two, tf_train_labels))
    
#     Optimize the Loss function. set learning rate as 0.5
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
#     Use the relu neural network
    train_prediction = tf.nn.softmax(logits_layer_two)
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights)+biases), weights_two)+biases_two)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights)+biases), weights_two)+biases_two)

Before we runt the training. Firstly, we set up standard for measuring the performance of our model. The Accuracy, in other word.

In [17]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Run the computation and find out the accuracy of our neural network model.

In [18]:
num_steps = 3001

with(tf.Session(graph=graph)) as sess:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = sess.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%\n" % accuracy(
            valid_prediction.eval(), valid_labels))
    print()
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 359.941528
Minibatch accuracy: 15.6%
Validation accuracy: 36.3%

Minibatch loss at step 500: 3.090946
Minibatch accuracy: 92.2%
Validation accuracy: 81.6%

Minibatch loss at step 1000: 2.286309
Minibatch accuracy: 89.1%
Validation accuracy: 82.3%

Minibatch loss at step 1500: 1.605369
Minibatch accuracy: 94.5%
Validation accuracy: 82.1%

Minibatch loss at step 2000: 0.258233
Minibatch accuracy: 96.1%
Validation accuracy: 83.0%

Minibatch loss at step 2500: 0.717077
Minibatch accuracy: 97.7%
Validation accuracy: 80.5%

Minibatch loss at step 3000: 0.125953
Minibatch accuracy: 97.7%
Validation accuracy: 83.5%


Test accuracy: 88.9%


**Problem: How about we change the number of neurons in the hidden layer?**